# This notebook will explain how to prepare different data sources for use with Lompe

This is how the data was prepared for the polar cap arc event on 15 December 2014 (notebook 06 in */Lompe_paper_figures/*).

We start by loading relevant modules:

In [ ]:
!pip install lompe viresclient
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from apexpy import Apex
import lompe
from lompe.model.cmodel import Cmodel
from lompe.data_tools import dataloader
from viresclient import set_token
from scipy.signal import butter, filtfilt, freqz
from viresclient import SwarmRequest
import aacgmv2
import warnings
warnings.filterwarnings('ignore')

In [ ]:
set_token(
        "https://vires.services/ows",
        set_default=True,
        token="kmxv5mTTyYwzw4kQ9lsCkGfQHtjjJRVZ",
    )  # key

We then select the event date and time we want to make Lompe data for. We specify the *'tempfile_path'* (location of processed data sets) and the *'basepath'* (loaction of 'raw' data files - the unprocessed data).

In [ ]:
# set time interval
event = '2022-12-19'
hour = 14
minute = 5
t0 = dt.datetime(int(event[0:4]), int(event[5:7]), int(event[8:10]), hour, minute)
print(t0)
DT = dt.timedelta(seconds = 5 * 60) # will select data from time +/- DT

# apex object for magnetic coordinates
a = Apex(t0.year)

# locations of data
tempfile_path = './sample_dataset/' # to put the processed data
basepath = tempfile_path + 'raw/' # unprocessed downloads

Note that if the already processed file for the event date exists in *'tempfile_path'* it will be used and the files from *'basepath'* are unused.

Then the cubed sphere grid is set up in the region we want to make a Lompe model. The region of interest should have a fairly even distribution of observations, and the grid resolution should be adjusted to fir the resolution of the observations (input data) (see Section 3.1 in the Lompe paper - [Laundal et al. 2022](https://doi.org/10.1029/2022JA030356)).

In [ ]:
position = (-114,73.454) # lon, lat
orientation = 75 # east, north
L, W = 4300e3, 4300e3 # dimensions of grid (L is along orientation vector)
Lres, Wres = 100.e3, 100.e3 # resolution of grid (Lres is along orientation vector)
grid = lompe.cs.CSgrid(lompe.cs.CSprojection(position, orientation), L, W, Lres, Wres, R = 6481.2e3)

You can plot the grid if you want to. This is a grid over a small region in North America:

In [ ]:
# plot grid and coastlines
fig, ax = plt.subplots(figsize = (10, 10))
ax.set_axis_off()
for lon, lat in grid.get_grid_boundaries():
    xi, eta = grid.projection.geo2cube(lon, lat)
    ax.plot(xi, eta, color = 'grey', linewidth = .4)

xlim, ylim = ax.get_xlim(), ax.get_ylim()
for cl in grid.projection.get_projected_coastlines():
    ax.plot(cl[0], cl[1], color = 'C0')

ax.set_xlim(xlim)
ax.set_ylim(ylim);

### Input data
##### DMSP SSUSI

First, we look at how images of the aurora can be used with Lompe. The following example will use the SSUSI image for the Polar Cap event (notebook 06) shown in the [Lompe paper](https://doi.org/10.1029/2022JA030356) in Figure 10. The SSUSI data to be used with these scripts can be downloaded from the [SSUSI homepage](https://ssusi.jhuapl.edu/) under the EDR_Aurora data product. The downloaded data is stored in netcdf files for each orbit. They should be stored in *'basepath'*. When constructing the functions returning Hall and Pedersen conductances inside the Lompe grid using the *lompe.model.cmodel.Cmodel()* class as done in the notebook '06_polar_cap_event', a helper function contained in *lompe.data_tools.dataloader.read_ssusi()* is used to read the downloaded data and put it into an xarray format. This processed file is saved in *'tempfile_path'*. When using *Cmodel*, SSUSI images are used as input data for auroral conductances. Further details on preparing the SSUSI based conductance functions are found in the *Cmodel()* class and *dataloader.read_ssusi()* function. Solar conductances are also included in the *Cmodel()* conductance model object (EUV = True).

The conductance model object can be constructed on the model grid:

In [ ]:
ssies = dataloader.read_ssusi(event, 'north', basepath, tempfile_path)

In [ ]:
from lompe.utils.conductance import hardy_EUV
Kp = 2 # for Hardy conductance model
SH = lambda lon = grid.lon, lat = grid.lat: hardy_EUV(lon, lat, Kp, t0, 'hall', F107=152    )
SP = lambda lon = grid.lon, lat = grid.lat: hardy_EUV(lon, lat, Kp, t0, 'pedersen', F107=152)
model = lompe.Emodel(grid, Hall_Pedersen_conductance = (SH, SP))



##### DMSP SSIES

Next, we show how ion drift meter data from the DMSP SSIES instrument can be obtained from the  [Madrigal database](http://cedar.openmadrigal.org/) and put into the Lompe format, again using tools in *lompe.data_tools.dataloader*.

*lompe.data_tools.dataloader.read_ssies()* fetches SSIES data using the MadrigalWeb module ([download module here](http://cedar.openmadrigal.org/madrigalDownload) or use [pip install](https://pypi.org/project/madrigalWeb/)). User information has to be provided, but user does not have to sign up anywhere beforehand. Specify which DMSP satellite you want data from (e.g. F16, F17, F18, or F19). *'basepath'* is used for temporary storage of downloaded files. The processed file will be saved in *'tempfile_path'*. If a processed file from the event date already exists in *'tempfile_path'* it will be used. *read_ssies()* will return the path to the processed file for the event date.

The *read_ssies()* function will fecth data for the entire event date. We select data from time interval of interest. The data far away from the model grid will be automatically excluded in Lompe. Then we make the Lompe data object.

##### SuperDARN
Due to its spatial coverage, a central data source to use with Lompe is SuperDARN convection measurements. Here, we show how the gridded data product produced with RST can be put into the Lompe inversion. This data product consists of merged data from all available radars in each hemisphere that has been spatially and temporally gridded. Such files can either be fetched from [Globus](https://www.globus.org/) after an invitation to join the 'SuperDARN mirror'. To get access, look here for a recipe: https://github.com/SuperDARNCanada/globus

Another source of such gridded data is the gridded published dataset by E.Thomas: https://doi.org/10.5281/zenodo.3618607 (2010-2016).

The gridmap file should be saved in *'basepath'*. The dataloader functions will read from there and save the processed data in *'tempfile_path'*. If a processed file from the event date already exists in *'tempfile_path'* it will be used. *read_sdarn()* will return the path to the processed file for the event date. data from the entire event date will be saved in the processed file, and we need to select data for the time interval of interest before creating the Lompe data object.

Note: Lompe data objects of the *'convection'* data type need the line-of-sight (LOS) parameter.

##### SuperMAG
Next is SuperMAG ground magnetometer data. The following function will read a netcdf file of all stations on the given day, downloaded from the [SuperMAG webpage](https://supermag.jhuapl.edu/). These files should be located in *'basepath'*. Note that the manual SuperMAG download has the download date as default filename, and if a local file is to be used the name of the file must be specified.

In [ ]:
file_name = 'all_stations_all2022.netcdf'
supermag = pd.read_hdf(dataloader.read_smag(event, basepath, tempfile_path, file_name))

Like the other read-functions, *read_smag()* creates a processed file for all available stations on the entire event date. You can remove stations with a latitude cutoff if you want to, however, data far from the model grid will automatically be removed in Lompe anyways. We must select the time interval of interest before we make the Lompe data object:

In [ ]:
# latitude cutoff of magnetometers

# select the time interval of interest
sm = supermag[t0 - DT : t0 + DT].dropna()

# make the Lompe data object
B = np.vstack((sm.Be.values, sm.Bn.values, sm.Bu.values))
coords = np.vstack((sm.lon.values, sm.lat.values))
# lompe.Data is the Lompe data format
#sm_data = lompe.Data(B * 1e-9, coords, datatype = 'ground_mag', scale = 100e-9)

sm_data = lompe.Data(B * 1e-9, coords, datatype = 'ground_mag', error = 10e-9, iweight = 1)

In [ ]:
def butter_bandpass(cutoffs, fs, order=4):
    if cutoffs[0] ==0:
        return butter(order, cutoffs[1], fs=fs, btype="low")
    else:
        return butter(order, cutoffs, fs=fs, btype="band")


def butter_bandpass_filter(data, cutoffs, fs, order=4):
    b, a = butter_bandpass(cutoffs, fs, order=order)
    y = filtfilt(b, a, data)
    return y

In [ ]:
def requester(sc_collection, measurement, residual, sampling_step=None, **kwargs):
        try:
            request = SwarmRequest()
            request.set_collection(sc_collection)
            if residual == True:
                request.set_products(
                    measurements=measurement,
                    models=["CHAOS"],
                    residuals=True,
                    sampling_step=sampling_step,
                )
            else:
                request.set_products(
                    measurements=measurement,
                    models=["CHAOS"],
                    sampling_step=sampling_step,
                )
            data = request.get_between((t0 - DT),(t0 + DT), **kwargs)
            df = data.as_dataframe()
        except:
            df = []
        return df

In [ ]:
def arrangement(time, array, shape):  # arranges B into a useable format for use later
    barranged = np.zeros((len(time), shape))
    # Re-arranges into proper (n x 3 ) matricies, ugly but works
    for j in range(len(time)):
        for k in range(shape):
            barranged[j][k] = array[j][k]
    return barranged

The Lompe Data objects of the 'ground_mag' data type do not have line-of-sight (LOS) parameters. The magnetic field measurements (here called *B*) has the (east, north, up) components integrated.

Lets do swarm now

In [ ]:
collectionE_02 = [
"SW_EXPT_EFIA_TCT02",
"SW_EXPT_EFIB_TCT02",
"SW_EXPT_EFIC_TCT02",
]

In [ ]:
from datetime import datetime, timedelta
from scipy.optimize import curve_fit
import geopack.geopack as gp
def footprint(time, latitude, longitude, altitude, alt,vsw= [-400,0,0]):
    """
    time, datetime, time for magnetic data for footprint
    vsw velocity of solar wind, tuple of x,y,z
    longitude of satellite in degrees
    latitude of satellite in degrees
    altitude of satellite in km from centre of earth (should be above ~6371)
    THIS CODE ONLY Works in the NOrthern hemisphere

    """
    def cubic(t, a, b, c, d):
            return a*t**3 + b*t**2 + c*t + d
    def x(t, params_x):
        return cubic(t, *params_x)

    def y(t, params_y):
        return cubic(t, *params_y)

    def z(t, params_z):
        return cubic(t, *params_z)
    def radius(t, params_x, params_y, params_z):
        return np.sqrt(x(t, params_x)**2 + y(t, params_y)**2 + z(t, params_z)**2)

    def curve_fit_func(xx,yy,zz, differencealt):

        r = np.linspace(1, 1.5, 100000)# construct an array of radiuses from 1-1.5

        radius_data=np.sqrt(xx**2+yy**2+zz**2)

        params_x, _ = curve_fit(cubic, radius_data, xx) #Constructs fits on the traces inward since the spatial resolution produced by geopack is limited.
        params_y, _ = curve_fit(cubic, radius_data, yy)
        params_z, _ = curve_fit(cubic, radius_data, zz)



        index_closest=np.argmin(np.abs(radius(r, params_x, params_y, params_z)-(alt-differencealt+6371)/6371))#Find the index that produces the closest radius to the altitude

        return x(r[index_closest],params_x ),y(r[index_closest],params_y ),z(r[index_closest],params_z )

    t1 = time
    t0 = datetime(1970,1,1) #epoch
    ut = (t1-t0).total_seconds()
    lat_sat=np.deg2rad(latitude)
    lon_sat=np.deg2rad(longitude) #converts to radii
    gp.recalc(ut)
    r, theta= gp.geodgeo(altitude,lat_sat,1) #this r accounts for earths oblateness, so we need to find the difference between my 6371 assumption and the real value and account for that
    differencearray= (altitude+6371)-r
    x_gc,y_gc,z_gc = gp.sphcar((r)/6371,theta,lon_sat,1)  #spherical to cartesian


    x_gsm, y_gsm, z_gsm = gp.geogsm(x_gc,y_gc,z_gc, 1) #cartesian to gsm

    x_foot,y_foot,z_foot=np.zeros(len(x_gsm)), np.zeros(len(y_gsm)), np.zeros(len(z_gsm)) #initalize an array
    for index in range(len(x_gsm)):
        x_foot_int, y_foot_int, z_foot_int, xx, _,zz = gp.trace(x_gsm[index], y_gsm[index], z_gsm[index], dir=1,rlim=1.6, maxloop=3000 ) #traces each set of lat,lon,alt outward

        _, _, _, xx2,yy2,zz2 = gp.trace(x_foot_int, y_foot_int, z_foot_int, dir=-1,rlim=100, maxloop=1000 )#Traces inward

        x_foot[index],y_foot[index],z_foot[index] = curve_fit_func(xx2,yy2,zz2, differencearray[index])






    x_done, y_done, z_done = gp.geogsm(x_foot, y_foot, z_foot, -1)

    alt_sat_done, lat_sat_done,lon_sat_done = np.zeros(len(x_done)), np.zeros(len(x_done)), np.zeros(len(x_done))
    for index in range(len(x_done)):

        r_done,theta_done,lon_sat_done[index]= gp.sphcar(x_done[index], y_done[index], z_done[index],-1)

        alt_sat_done[index], lat_sat_done[index]= gp.geodgeo(r_done*6371,theta_done,-1) #TODO check if this is right



    if np.any(np.abs(alt_sat_done - alt) > 5):
        raise Exception("One or more values in the footprinting are greater than 5km away from the specified alt. Contact owner for a fix, not your fault")

    sat_lla=np.array([ np.rad2deg(lon_sat_done)-360,np.rad2deg(lat_sat_done), alt_sat_done])
    return sat_lla

In [ ]:
collectionB_01 = ["SW_OPER_MAGA_LR_1B", "SW_OPER_MAGB_LR_1B", "SW_OPER_MAGC_LR_1B"]
swarmmags=[]
alt=150
BUsed_swarmb=[]
coords_swarmb=[]
for i in range(3):
    dsB = requester(
        collectionB_01[i],
        "B_NEC",
        True,
        asynchronous=False,
        show_progress=False,
    )


    Bdata = dsB["B_NEC_res_CHAOS"]  # data package
    time = Bdata.index.to_numpy()
    Bdata = Bdata.to_numpy()

    barranged = arrangement(time, Bdata, 3)


    coords= footprint( t0,dsB['Latitude'].to_numpy(), dsB['Longitude'].to_numpy(), (dsB["Radius"].to_numpy()-6.371e6)/1e3, alt)
    coords[2] = dsB["Radius"].to_numpy() #After talking to Karl, keep altitude but change lat and lon appropriately

    Bused=np.array([ barranged[:,1],barranged[:,0], -1*barranged[:,2]])
    plt.plot(coords[1], Bused[0], label='unfilt')
    bfilter = np.zeros(np.shape(Bused))

    plt.plot(coords[1], Bused[0], label='filt')
    if i==1:
        BUsed_swarmb.append(Bused)
        coords_swarmb.append(coords)

    swarmmags.append(lompe.Data(Bused*1e-9,coords, datatype='space_mag_full', error=5e-9,iweight=1.0))

plt.legend()

##### Iridium

Data handling of *Iridium* space magnetometer data (provided by [AMPERE](http://ampere.jhuapl.edu/)) is also possible by using *dataloader.read_iridium()*. The raw data for the event date - AMPERE dB (Raw Vectors) - with a 2 minute time step  be downloaded from [this page](http://ampere.jhuapl.edu/dataraw/index.html) and should be saved in *'basepath'*. *read_iridium()* will save the processed file in *'tempfile_path'* and return the loaction of the file. As for the other data sets, we must select the time interval of interest before we make the Lompe data object.

Unfortunately, space magnetometer data from AMPERE is not available for the 15 December 2014 event. For the sake of making the data loading example complete, we will use an example event time 5 April 2012 which is a date where we have data. This data will naturally not be included in the 15 December 2014 model.

In [ ]:

example_event = '2022-12-19'
# get the processed data set
iridium = pd.read_hdf(dataloader.read_iridium(example_event, basepath, tempfile_path))

# select the time interval of interest
irid = iridium[(iridium.time >= t0) & (iridium.time <= t0 + DT)]

# make the Lompe data object
irid_B = np.vstack((irid.B_e.values, irid.B_n.values, irid.B_r.values))
irid_coords = np.vstack((irid.lon.values, irid.lat.values, irid.r.values))
# lompe.Data is the Lompe data format
#iridium_data = lompe.Data(irid_B * 1e-9, irid_coords, datatype = 'space_mag_fac', scale = 200e-9)

iridium_data = lompe.Data(irid_B * 1e-9, irid_coords, datatype = 'space_mag_fac', error = 70e-9, iweight=1)

##### Data type for Lompe
When making the Lompe data objects, the 'datatype' parameter must be specified. The options are:

- **'ground_mag'**: Magnetic field perturbations on ground (given in Tesla). This is what was used for the SuperMAG observations.

- **'space_mag_full'**: Magnetic field perturbations in space associated with field-aligned currents **and** horizontal divergence-free currents below the satellite (in Tesla). Typically used for low-flying precise magnetometers (e.g. Swarm and CHAMP).

- **'space_mag_fac'**: Magnetic field perturbations in space associated with field-aligned currents (in Tesla). Magnetic field in space that corresponds to only the field-aligned currents. For example, Iridium magnetic data is dominated by FACs, since it is taken at around 800 km altitude, and by magnetometers that do not have the precision of science-mission instruments.

- **'convection'**: Ionospheric convection velocity perpendicular to the magnetic field, mapped to the ionospheric radius (in meters per second). The line-of-sight direction must be specified using the LOS keyword (that should contain the eastward and northward components of the line-of-sight vector). This is the data type for both the ground-based SuperDARN observations and the space-based observations from DMSP SSIES.

- **'Efield'**: Ionospheric convection electric field, perpendicular to B and mapped to the ionospheric radius (in Volts per meter). The LOS keyword can be used for this parameter also, if only one component of the electric field is known.

- **'fac'**: Field-aligned electric current density (in Ampere per meter$^2$). This parameter is only meant to be used with large-scale datasets or simulation output that can be interpolated to the Lompe model grid. This is different from all the other datatypes used in Lompe.

##### 'scale' parameter for the Lompe data object (DEPRECATED)
The scale keyword determines a weight for the dataset. It should be set to a typical order of magnitude for the measurement. For example, magnetometer data have typical magnitude $10^{-7}$ T, and plasma flow data typically $100$ m/s. The scale parameter determines the relative importance of the dataset in the model-data misfit minimization. Increasing it will reduce importance of the data set. If you only have one dataset, the scale parameter should be irrelevant. The scale parameter is described in Section 3.3 in the [Lompe paper](https://doi.org/10.1029/2022JA030356) ($\sigma_B$, $\sigma_E$ etc.)

##### 'iweight' and 'error' parameters for the Lompe data object
As of June 2023, the 'iweight' (importance weight) and 'error' keywords should be used instead of the 'scale' keyword. `error` should be set to some reasonable estimate of the uncertainty of the measurements (all measurements have uncertainty!). `iweight` is subjective and is determined by the user; see the documentation in lompe.model.data.

### Final electric field model
Now the data can be added to Lompe and the inversion can be performed. Note that the Lompe electric field model is given Hall and Pedersen conductances from the conductance model when initialized.

In [ ]:
# initialize Lompe
model = lompe.Emodel(grid, Hall_Pedersen_conductance = (SH, SP))

# add data
#model.add_data(iridium_data)
model.add_data(sm_data)
#model.add_data(swarmmags[0],swarmmags[1], swarmmags[2] , )

# run inversion
model.run_inversion(l1 = 1, l2 = 10)
# l1 and l2 are regularization parameters that control the damping of
# 1) model norm, and 2) gradient of SECS amplitudes (charges) in magnetic eastward direction

Note that multiple Lompe Data objects of the same data type can be passed to the Lompe model once it's initialized.

You can use *lompeplot* to get a summary figure with the model output. When 'include_data = True', the location of input data is shown as orange arrows. This is a similar figure to the one created in notebook 06 with slightly different data input (no DMSP F18 input and including SuperMAG observations).

In [ ]:
fig = lompe.lompeplot(model, include_data = True, time = t0, apex = a,
                      colorscales = {'fac'        : np.linspace(-0.55, 0.55, 40) * 1e-6 * 2,
                                     'ground_mag' : np.linspace(-380, 380, 50) * 1e-9 / 3, # upward component
                                     'hall'       : np.linspace(0, 6, 32), # mho
                                     'pedersen'   : np.linspace(0, 6, 32)}, # mho
                      quiverscales={'ground_mag'      : 1000*1e-9,
                                    'space_mag_fac'   : 1000*1e-9,
                                    'space_mag_full'  : 1000*1e-9,
                                    'electric_current': 1000 * 1e-3}, return_axes=True)

fig[1][1][0].cla()
fig[1][1][0].scatter(-114,65, s=100)

In [ ]:
# initialize Lompe
model = lompe.Emodel(grid, Hall_Pedersen_conductance = (SH, SP))

# add data
model.add_data(iridium_data)
model.add_data(sm_data)
#model.add_data(swarmmags[0],swarmmags[1], swarmmags[2] , )

# run inversion
model.run_inversion(l1 = 1, l2 = 10)
# l1 and l2 are regularization parameters that control the damping of
# 1) model norm, and 2) gradient of SECS amplitudes (charges) in magnetic eastward direction

In [ ]:
fig = lompe.lompeplot(model, include_data = True, time = t0, apex = a,
                      colorscales = {'fac'        : np.linspace(-0.55, 0.55, 40) * 1e-6 * 2,
                                     'ground_mag' : np.linspace(-380, 380, 50) * 1e-9 / 3, # upward component
                                     'hall'       : np.linspace(0, 6, 32), # mho
                                     'pedersen'   : np.linspace(0, 6, 32)}, # mho
                      quiverscales={'ground_mag'      : 1000*1e-9,
                                    'space_mag_fac'   : 1000*1e-9,
                                    'space_mag_full'  : 1000*1e-9,
                                    'electric_current': 1000 * 1e-3}, return_axes=True)


In [ ]:
# initialize Lompe
model = lompe.Emodel(grid, Hall_Pedersen_conductance = (SH, SP))

# add data
#model.add_data(iridium_data)
#model.add_data(sm_data)
model.add_data(swarmmags[1])

# run inversion
model.run_inversion(l1 = 1, l2 = 10)
# l1 and l2 are regularization parameters that control the damping of
# 1) model norm, and 2) gradient of SECS amplitudes (charges) in magnetic eastward direction

In [ ]:
fig = lompe.lompeplot(model, include_data = True, time = t0, apex = a,
                      colorscales = {'fac'        : np.linspace(-0.55, 0.55, 40) * 1e-6 * 2,
                                     'ground_mag' : np.linspace(-380, 380, 50) * 1e-9 / 3, # upward component
                                     'hall'       : np.linspace(0, 6, 32), # mho
                                     'pedersen'   : np.linspace(0, 6, 32)}, # mho
                      quiverscales={'ground_mag'      : 1000*1e-9,
                                    'space_mag_fac'   : 1000*1e-9,
                                    'space_mag_full'  : 1000*1e-9,
                                    'electric_current': 1000 * 1e-3}, return_axes=True)


In [ ]:
fig, ax = plt.subplots( dpi=300)
lompe.visualization.plot_quiver(  ax, model, 'space_mag_fac')
lompe.visualization.plot_contour( ax, model, 'fac')


lompe.visualization.plot_datasets(ax, model, 'space_mag_fac')
lompe.visualization.plot_datasets(ax, model, 'space_mag_full')
ax.set_title('FAC and magnetic field')


lon = np.linspace(-123, -105, 200)
lat= np.linspace(62.3, 62.45, 200)



x, y= model.grid_J.projection.geo2cube(lon, lat)
scat = ax.plot(x, y, color='purple', label='Location of Arc Based on Ewogram')
x, y= model.grid_J.projection.geo2cube( -114.3718, 62.4540)
lon = np.linspace(-123, -105, 200)
lat= np.linspace(62.5, 63.6, 200)


x, y= model.grid_J.projection.geo2cube(lon, lat)
scat = ax.plot(x, y, color='purple')


plt.legend()

In [ ]:
fig, ax = plt.subplots( dpi=300)
lompe.visualization.plot_quiver(  ax, model, 'space_mag_fac')
lompe.visualization.plot_contour( ax, model, 'fac')


lompe.visualization.plot_datasets(ax, model, 'space_mag_fac')
lompe.visualization.plot_datasets(ax, model, 'space_mag_full')
ax.set_title('FAC and magnetic field')


lon = np.linspace(-123, -105, 200)
lat= np.linspace(62.3, 62.45, 200)



x, y= model.grid_J.projection.geo2cube(lon, lat)
scat = ax.plot(x, y, color='purple', label='Location of Arc Based on Ewogram')
x, y= model.grid_J.projection.geo2cube( -114.3718, 62.4540)
lon = np.linspace(-123, -105, 200)
lat= np.linspace(62.5, 63.6, 200)


x, y= model.grid_J.projection.geo2cube(lon, lat)
scat = ax.plot(x, y, color='purple')

plt.xlim(-0.4,-0.1)
plt.ylim(0,0.4)

plt.legend()

In [ ]:
# initialize Lompe
model = lompe.Emodel(grid, Hall_Pedersen_conductance = (SH, SP))

# add data
model.add_data(iridium_data)
model.add_data(sm_data)
model.add_data(swarmmags[0],swarmmags[1], swarmmags[2] , )

# run inversion
model.run_inversion(l1 = 1, l2 = 10)
# l1 and l2 are regularization parameters that control the damping of
# 1) model norm, and 2) gradient of SECS amplitudes (charges) in magnetic eastward direction

In [ ]:
fig = lompe.lompeplot(model, include_data = True, time = t0, apex = a,
                      colorscales = {'fac'        : np.linspace(-0.55, 0.55, 40) * 1e-6 * 2,
                                     'ground_mag' : np.linspace(-380, 380, 50) * 1e-9 / 3, # upward component
                                     'hall'       : np.linspace(0, 6, 32), # mho
                                     'pedersen'   : np.linspace(0, 6, 32)}, # mho
                      quiverscales={'ground_mag'      : 1000*1e-9,
                                    'space_mag_fac'   : 1000*1e-9,
                                    'space_mag_full'  : 1000*1e-9,
                                    'electric_current': 1000 * 1e-3}, return_axes=True)


In [ ]:
# initialize Lompe
model = lompe.Emodel(grid, Hall_Pedersen_conductance = (SH, SP))

# add data
model.add_data(iridium_data)
model.add_data(sm_data)
model.add_data(swarmmags[0],swarmmags[1], swarmmags[2] , )


# run inversion
model.run_inversion(l1 = 1, l2 = 10)
# l1 and l2 are regularization parameters that control the damping of
# 1) model norm, and 2) gradient of SECS amplitudes (charges) in magnetic eastward direction

In [ ]:
fig = lompe.lompeplot(model, include_data = True, time = t0, apex = a,
                      colorscales = {'fac'        : np.linspace(-0.55, 0.55, 40) * 1e-6 * 2,
                                     'ground_mag' : np.linspace(-380, 380, 50) * 1e-9 / 3, # upward component
                                     'hall'       : np.linspace(0, 6, 32), # mho
                                     'pedersen'   : np.linspace(0, 6, 32)}, # mho
                      quiverscales={'ground_mag'      : 1000*1e-9,
                                    'space_mag_fac'   : 1000*1e-9,
                                    'space_mag_full'  : 2000*1e-9,
                                    'electric_current': 1000 * 1e-3}, return_axes=True)


In [ ]:

dataloader.read_sdarn(event, basepath, tempfile_path)
superdarn = pd.read_hdf(dataloader.read_sdarn(event, basepath, tempfile_path))

# select the time interval of interest (also remove very high speeds)
sd = superdarn.loc[(superdarn.index >= t0 - DT) & (superdarn.index <= t0 + DT) & (superdarn.vlos < 2000)].dropna()

# make the Lompe data object
vlos = sd['vlos'].values
coords = np.vstack((sd['glon'].values, sd['glat'].values))
los  = np.vstack((sd['le'].values, sd['ln'].values))
# lompe.Data is the Lompe data format
#sd_data = lompe.Data(vlos, coordinates = coords, LOS = los, datatype = 'convection', scale = 500 )

sd_data = lompe.Data(vlos, coordinates = coords, LOS = los, datatype = 'convection', error = 50, iweight=1 )

In [ ]:
# initialize Lompe
model = lompe.Emodel(grid, Hall_Pedersen_conductance = (SH, SP))

# add data
#model.add_data(iridium_data)
#model.add_data(sm_data)
#model.add_data(swarmmags[0],swarmmags[1], swarmmags[2] , )

model.add_data(sd_data)


# run inversion
model.run_inversion(l1 = 1, l2 = 10)
# l1 and l2 are regularization parameters that control the damping of
# 1) model norm, and 2) gradient of SECS amplitudes (charges) in magnetic eastward direction

In [ ]:
fig = lompe.lompeplot(model, include_data = True, time = t0, apex = a,
                      colorscales = {'fac'        : np.linspace(-0.55, 0.55, 40) * 1e-6 * 2,
                                     'ground_mag' : np.linspace(-380, 380, 50) * 1e-9 / 3, # upward component
                                     'hall'       : np.linspace(0, 6, 32), # mho
                                     'pedersen'   : np.linspace(0, 6, 32)}, # mho
                      quiverscales={'ground_mag'      : 1000*1e-9,
                                    'space_mag_fac'   : 2000*1e-9,
                                    'space_mag_full'  : 2000*1e-9,
                                    'electric_current': 1000 * 1e-3}, return_axes=True)


In [ ]:
# initialize Lompe
model = lompe.Emodel(grid, Hall_Pedersen_conductance = (SH, SP))

# add data
model.add_data(iridium_data)
model.add_data(sm_data)
model.add_data(swarmmags[0],swarmmags[1], swarmmags[2] , )


# run inversion
model.run_inversion(l1 = 1, l2 = 10)
# l1 and l2 are regularization parameters that control the damping of
# 1) model norm, and 2) gradient of SECS amplitudes (charges) in magnetic eastward direction

In [ ]:
fig = lompe.lompeplot(model, include_data = True, time = t0, apex = a,
                      colorscales = {'fac'        : np.linspace(-0.55, 0.55, 40) * 1e-6 * 2,
                                     'ground_mag' : np.linspace(-380, 380, 50) * 1e-9 / 3, # upward component
                                     'hall'       : np.linspace(0, 6, 32), # mho
                                     'pedersen'   : np.linspace(0, 6, 32)}, # mho
                      quiverscales={'ground_mag'      : 1000*1e-9,
                                    'space_mag_fac'   : 2000*1e-9,
                                    'space_mag_full'  : 500*1e-9,
                                    'electric_current': 1000 * 1e-3}, return_axes=True)


In [ ]:
def unit_array(array):
    arraysum = np.sum(np.abs(array), axis=1)
    # Normalizes and finds unitary
    array_unit = array / arraysum[:, np.newaxis]  # normalizes
    return array_unit

In [ ]:

def find_closest_indices(times1, times2):
    # Convert to numpy arrays
    times1 = np.array(times1)
    times2 = np.array(times2)

    # Compute the differences between each time in times1 and all times in times2
    # Resulting in a 2D array where each row contains the absolute differences for one time in times1
    differences = np.abs(times1[:, None] - times2)

    # Find the index of the minimum difference for each time in times1
    closest_indices = np.argmin(differences, axis=1)

    return closest_indices

In [ ]:
#Swarm A's magnetics are not great but swarm B's are okay, we use this to our advantage to increase our resolution in E

measurements_E = [
        "VsatN",
        "VsatE",
        "VsatC",
        "Vixv",
        "Viy",
        "Viz",
        "Quality_flags",
        'Bx',
        'By',
        'Bz'
    ]
ds = requester(
    "SW_EXPT_EFIB_TCT02", #Mag B, high resolution, 50Hz B (Magnetic field)
    measurements_E, #Magnetic field in NEC coordinates
    True,
    asynchronous=False,
    show_progress=False)
dsB = requester(
    'SW_OPER_MAGB_HR_1B', #Mag B, high resolution, 50Hz B (Magnetic field)
    ["q_NEC_CRF"], #Magnetic field in NEC coordinates
    False,
    asynchronous=False,
    show_progress=False)
Etime=ds.index


In [ ]:
from scipy.spatial.transform import Rotation as R
import numpy as np

def quaternion_inverse_scipy(q):
    # Ensure q is a numpy array
    q = np.asarray(q)

    # Create a Rotation object from the quaternion
    rotation = R.from_quat(q)  # Note: scipy uses [x, y, z, w] format

    # Compute the inverse rotation
    inverse_rotation = rotation.inv()


    return inverse_rotation

In [ ]:
indicies=find_closest_indices(ds.index, dsB.index)
quatnecrf=dsB["q_NEC_CRF"].to_numpy()[indicies]
quaternions = []
vsat=np.array(ds["Viy"])

coords_E= footprint( t0,ds['Latitude'].to_numpy(), ds['Longitude'].to_numpy(), (ds["Radius"].to_numpy()-6.371e6)/1e3, alt )[:-1]
vused=vsat

vused = np.where((vused > 3500) | (vused < -3500), np.nan, vused)

plt.legend()
print(ds["VsatN"].to_numpy()/np.sqrt(ds["VsatN"].to_numpy()**2+ ds["VsatE"].to_numpy()**2))
LOS=np.array([ds["VsatN"].to_numpy()/np.sqrt(ds["VsatN"].to_numpy()**2+ ds["VsatE"].to_numpy()**2),
 ds["VsatE"].to_numpy()/np.sqrt(ds["VsatN"].to_numpy()**2+ ds["VsatE"].to_numpy()**2)])

Eswarm = lompe.Data(vused, coordinates = coords_E, datatype = 'convection', error = 10, iweight=1.0, LOS=LOS)

![image.png](attachment:image.png)

![image.png](attachment:image.png) VELOCITIES Are westward, so negative V, however, lompe keeps putting negative = eastward

In [ ]:
# initialize Lompe
model = lompe.Emodel(grid, Hall_Pedersen_conductance = (SH, SP))

# add data
#model.add_data(iridium_data)
#model.add_data(sm_data)
#model.add_data(swarmmags[0],swarmmags[1], swarmmags[2] , )
#
#model.add_data(sd_data)
model.add_data(Eswarm)


# run inversion
model.run_inversion(l1 = 1, l2 = 10)
# l1 and l2 are regularization parameters that control the damping of
# 1) model norm, and 2) gradient of SECS amplitudes (charges) in magnetic eastward direction

In [ ]:
fig = lompe.lompeplot(model, include_data = True, time = t0, apex = a,
                      colorscales = {'fac'        : np.linspace(-0.55, 0.55, 40) * 1e-6 * 2,
                                     'ground_mag' : np.linspace(-380, 380, 50) * 1e-9 / 3, # upward component
                                     'hall'       : np.linspace(0, 6, 32), # mho
                                     'pedersen'   : np.linspace(0, 6, 32)}, # mho
                      quiverscales={'ground_mag'      : 500*1e-9,
                                    'space_mag_fac'   : 2000*1e-9,
                                    'space_mag_full'  : 500*1e-9,
                                    'electric_current': 1000* 1e-3,
                                    'convection': 6000
                                    }, return_axes=True)


In [ ]:
# initialize Lompe
model = lompe.Emodel(grid, Hall_Pedersen_conductance = (SH, SP))

# add data
#model.add_data(iridium_data)
#model.add_data(sm_data)
model.add_data(swarmmags[0],swarmmags[1], swarmmags[2] , )
#
#model.add_data(sd_data)
model.add_data(Eswarm)


# run inversion
model.run_inversion(l1 = 1, l2 = 10)
# l1 and l2 are regularization parameters that control the damping of
# 1) model norm, and 2) gradient of SECS amplitudes (charges) in magnetic eastward direction

In [ ]:


fig = lompe.lompeplot(model, include_data = True, time = t0, apex = a,
                      colorscales = {'fac'        : np.linspace(-0.5, 0.5, 40) * 1e-6 * 2,
                                     'ground_mag' : np.linspace(-380, 380, 50) * 1e-9 / 3, # upward component
                                     'hall'       : np.linspace(0, 6, 32), # mho
                                     'pedersen'   : np.linspace(0, 6, 32)}, # mho
                      quiverscales={'ground_mag'      : 600*1e-9,
                                    'space_mag_fac'   : 2000*1e-9,
                                    'space_mag_full'  : 2000*1e-9,
                                    'electric_current': 1000* 1e-3,
                                    'convection': 7000,
                                    })

xeas=fig.get_axes()



In [ ]:
# initialize Lompe
model = lompe.Emodel(grid, Hall_Pedersen_conductance = (SH, SP))

# add data
model.add_data(iridium_data)
model.add_data(sm_data)
model.add_data(swarmmags[0],swarmmags[1], swarmmags[2] , )

model.add_data(sd_data)
model.add_data(Eswarm)


# run inversion
model.run_inversion(l1 =2, l2 = 10)
# l1 and l2 are regularization parameters that control the damping of
# 1) model norm, and 2) gradient of SECS amplitudes (charges) in magnetic eastward direction

In [ ]:


fig = lompe.lompeplot(model, include_data = True, time = t0, apex = a,
                      colorscales = {'fac'        : np.linspace(-0.8, 0.8, 40) * 1e-6 * 2,
                                     'ground_mag' : np.linspace(-380, 380, 50) * 1e-9 / 3, # upward component
                                     'hall'       : np.linspace(0, 6, 32), # mho
                                     'pedersen'   : np.linspace(0, 6, 32)}, # mho
                      quiverscales={'ground_mag'      : 600*1e-9,
                                    'space_mag_fac'   : 1200*1e-9,
                                    'space_mag_full'  : 1200*1e-9,
                                    'electric_current': 1000* 1e-3,
                                    'convection': 5000,
                                    })

xeas=fig.get_axes()



In [ ]:
fig, ax = plt.subplots( dpi=300)
print(ax)
lompe.visualization.plot_potential(ax, model)
print(lompe.visualization.QUIVERSCALES)
lompe.visualization.plot_quiver(ax, model, 'convection')
lompe.visualization.plot_datasets(ax, model, 'convection')

lon = np.linspace(-123, -105, 200)
lat= np.linspace(62.3, 62.45, 200)



x, y= model.grid_J.projection.geo2cube(lon, lat)
scat = ax.plot(x, y, color='purple', label='Location of Arc Based on Ewogram')
x, y= model.grid_J.projection.geo2cube( -114.3718, 62.4540)
lon = np.linspace(-123, -105, 200)

scat = ax.plot(x, y, color='purple')

plt.legend()
lompe.visualization.plot_mlt(ax, model, time[0], a, np.array([0,3,6,9,12]), color = 'grey', alpha=0.3)

In [ ]:
fig, ax = plt.subplots( dpi=300)
lompe.visualization.plot_quiver(  ax, model, 'space_mag_fac')
lompe.visualization.plot_contour( ax, model, 'fac')
lon = np.linspace(-123, -105, 200)
lat= np.linspace(62.3, 62.45, 200)


lompe.visualization.plot_datasets(ax, model, 'space_mag_full')
lompe.visualization.plot_datasets(ax, model, 'space_mag_fac')

x, y= model.grid_J.projection.geo2cube(lon, lat)
scat = ax.plot(x, y, color='purple', label='Location of Arc Based on Ewogram')
x, y= model.grid_J.projection.geo2cube( -114.3718, 62.4540)
lon = np.linspace(-123, -105, 200)
lat= np.linspace(62.5, 63.6, 200)
x, y= model.grid_J.projection.geo2cube(lon, lat)
scat = ax.plot(x, y, color='purple')



In [ ]:
fig = plt.figure(figsize=(10, 10), dpi=300)  # Adjust figure size as needed
import matplotlib.gridspec as gridspec
# Define GridSpec with 1 column on the left and 3 stacked rows on the right
gs = gridspec.GridSpec(2, 2, figure=fig, wspace=0.1, hspace=0.125, height_ratios=[1,0.05])  # 3 rows, 4 columns for flexibility
ax1 = fig.add_subplot(gs[0, 0])
ax2 = fig.add_subplot(gs[0, 1])
ax3 = fig.add_subplot(gs[1, :])
plt.setp(ax1.get_xticklabels(), visible=False)
plt.setp(ax1.get_yticklabels(), visible=False)
plt.setp(ax2.get_xticklabels(), visible=False)
plt.setp(ax2.get_yticklabels(), visible=False)
lompe.visualization.plot_potential(ax1, model)
print(lompe.visualization.QUIVERSCALES)
lompe.visualization.plot_quiver(ax1, model, 'convection')
lompe.visualization.plot_datasets(ax1, model, 'convection')

lon = np.linspace(-123, -105, 200)
lat= np.linspace(62.3, 62.45, 200)



x, y= model.grid_J.projection.geo2cube(lon, lat)
scat = ax1.plot(x, y, color='purple', label='Location of Arc Based on Ewogram')
x, y= model.grid_J.projection.geo2cube( -114.3718, 62.4540)
lon = np.linspace(-123, -105, 200)

scat = ax1.plot(x, y, color='purple')


lompe.visualization.plot_mlt(ax1, model, time[0], a, np.array([0,3,6,9,12]), color = 'grey', alpha=0.8)

ax1.set_title("Convection Velocity and Electric Potential")
ax2.set_title("FAC and Magnetic Field")

lompe.visualization.plot_quiver(  ax2, model, 'space_mag_fac')
lompe.visualization.plot_contour( ax2, model, 'fac')
lon = np.linspace(-123, -105, 200)
lat= np.linspace(62.3, 62.45, 200)


lompe.visualization.plot_datasets(ax2, model, 'space_mag_full')
lompe.visualization.plot_datasets(ax2, model, 'space_mag_fac')

x, y= model.grid_J.projection.geo2cube(lon, lat)
scat = ax2.plot(x, y, color='purple', label='Location of Arc Based on Ewogram')
x, y= model.grid_J.projection.geo2cube( -114.3718, 62.4540)
lompe.visualization.plot_mlt(ax2, model, time[0], a, np.array([0,3,6,9,12]), color = 'grey', alpha=0.8)

fac_levels=np.linspace(-0.8, 0.8, 40) * 1e-6 * 2

xx = np.vstack((fac_levels, fac_levels)) * 1e6
yy = np.vstack((np.zeros_like(fac_levels), np.ones_like(fac_levels)))
ax3.contourf(xx, yy, xx, cmap = plt.cm.bwr, levels = fac_levels * 1e6)
ax3.set_xlabel('$\mu$A/m$^2$')
ax3.set_yticks([])

In [ ]:
fig, ax = plt.subplots( dpi=300)
lompe.visualization.plot_quiver(  ax, model, 'space_mag_fac')
lompe.visualization.plot_contour( ax, model, 'fac')


lompe.visualization.plot_datasets(ax, model, 'space_mag_fac', alpha=0.5)
lompe.visualization.plot_datasets(ax, model, 'space_mag_full', alpha=0.5)

lat, lon = coords_swarmb[0][1], coords_swarmb[0][0]+0.3
print(lat)
x, y= model.grid_J.projection.geo2cube(lon, lat)
from matplotlib import cm
print((Bused[0]))


lon, lat = coords_E
x, y= model.grid_J.projection.geo2cube(lon, lat)
from matplotlib import cm

masked_array = np.ma.array(vused[0], mask=np.isnan(vused[0]))

#ax.plot(x,y, c=cm.viridis(masked_array))

ax.set_title('FAC and magnetic field')



lon = np.linspace(-123, -105, 200)
lat= np.linspace(62.3, 62.45, 200)


x, y= model.grid_J.projection.geo2cube(lon, lat)
scat = ax.plot(x, y, color='purple', label='Location of Arc Based on Ewogram')
x, y= model.grid_J.projection.geo2cube( -114.3718, 62.4540)


lompe.visualization.plot_mlt(ax, model, time[0], a, np.array([0,3,6,9,12]), color = 'grey', alpha=0.3)

#lompe.visualization.plot_datasets(ax, model, 'convection', color='green')
plt.legend()


In [ ]:
plt.plot( coords_swarmb[0][1], BUsed_swarmb[0][0])

In [ ]:
fig, ax = plt.subplots( dpi=300)
lompe.visualization.plot_contour(ax, model, 'hall')
lompe.visualization.plot_coastlines(ax, model, color = 'grey')
lompe.visualization.plot_mlt(ax, model, time, a, color = 'grey')
lons, lats=np.linspace(-140,-90, 100), [62.4]*100 #Lat of Trex

lon, lat = lons, lats #Lat of Trex
x, y= model.grid_J.projection.geo2cube(lon, lat)
scat = ax.plot(x, y, color='red')

lons, lats=np.linspace(-140,-90, 100), [62.83]*100 #Lat of Trex

lon, lat = lons, lats #Lat of Trex
x, y= model.grid_J.projection.geo2cube(lon, lat)
scat = ax.plot(x, y, color='red', label='location of Velocity Shear')

lons, lats=np.linspace(-140,-90, 100), [62.3]*100 #Lat of Trex

lon, lat = lons, lats #Lat of Trex
x, y= model.grid_J.projection.geo2cube(lon, lat)
scat = ax.plot(x, y, color='purple')

lons, lats=np.linspace(-140,-90, 100), [62.86]*100 #Lat of Trex

lon, lat = lons, lats #Lat of Trex
x, y= model.grid_J.projection.geo2cube(lon, lat)
scat = ax.plot(x, y, color='purple', label='Location of Arc Based on Keogram')
ax.set_title('Hall conductance')

In [ ]:
fig, ax = plt.subplots( dpi=300)
lompe.visualization.plot_contour(ax, model, 'pedersen')
lompe.visualization.plot_coastlines(ax, model, color = 'grey')
lons, lats=np.linspace(-140,-90, 100), [62.4]*100 #Lat of Trex

lon, lat = lons, lats #Lat of Trex
x, y= model.grid_J.projection.geo2cube(lon, lat)
scat = ax.plot(x, y, color='red')

lons, lats=np.linspace(-140,-90, 100), [62.83]*100 #Lat of Trex

lon, lat = lons, lats #Lat of Trex
x, y= model.grid_J.projection.geo2cube(lon, lat)
scat = ax.plot(x, y, color='red', label='location of Velocity Shear')

lons, lats=np.linspace(-140,-90, 100), [62.3]*100 #Lat of Trex

lon, lat = lons, lats #Lat of Trex
x, y= model.grid_J.projection.geo2cube(lon, lat)
scat = ax.plot(x, y, color='purple')

lons, lats=np.linspace(-140,-90, 100), [62.86]*100 #Lat of Trex

lon, lat = lons, lats #Lat of Trex
x, y= model.grid_J.projection.geo2cube(lon, lat)
scat = ax.plot(x, y, color='purple', label='Location of Arc Based on Keogram')

ax.set_title('Pederson conductance')

The top row shows, from left to right (input data is shown in orange):
- Convection flow field and electric potential contours
- horizontal magnetic field disturbances 110 km above the ionosphere as black arrows and radial current density as color contours
- horizontal ground magnetic field perturbations as black arrows and radial magnetic field perturbations as color contours
- a map that shows the grid’s position and orientation with respect to apex magnetic latitude and local time.

The bottom row shows, from left to right:
- Pedersen conductance
- Hall conductance
- horizontal height-integrated ionospheric currents based on Lompe output
- color scale / vector scales